In [ ]:
!wget http://visual.ic.uff.br/proeng/thiagoelias/database.rar

import os,glob
import numpy as np
import cv2
import glob
import pickle
import tensorflow as tf
import argparse
import re
import datetime
from skimage.transform import rescale, resize, downscale_local_mean
from tensorflow.keras.layers import  Input,Conv2D,BatchNormalization,Activation,Subtract,LeakyReLU,Add,Average,Lambda,MaxPool2D,Dropout,UpSampling2D,Concatenate,Multiply,GlobalAveragePooling2D,Dense,ZeroPadding2D,AveragePooling2D
from tensorflow.keras.layers import concatenate,Flatten,ConvLSTM2D,MaxPooling2D,ReLU,Conv2DTranspose
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skimage.feature import hog,local_binary_pattern
from skimage import data, exposure
from skimage.transform import radon, rescale
from skimage.filters import roberts, sobel, scharr, prewitt
from skimage import feature

!pip install rarfile

--2021-07-07 19:24:28--  http://visual.ic.uff.br/proeng/thiagoelias/database.rar
Resolving visual.ic.uff.br (visual.ic.uff.br)... 200.20.15.38
Connecting to visual.ic.uff.br (visual.ic.uff.br)|200.20.15.38|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 483263536 (461M)
Saving to: ‘database.rar’

database.rar        100%[===================>] 460.88M  4.43MB/s    in 81s     

2021-07-07 19:25:49 (5.71 MB/s) - ‘database.rar’ saved [483263536/483263536]



In [ ]:
string = [138,179,180,181,192,198,202,204,209,210,213,240,241,245,249,250,251,255,257,259,261,263,264,270,400,450,500,550,600]
affected_files_text = []
for f in string:
    list1 = []
    normal_dir = "../input/thermal-images-for-breast-cancer-diagnosis-dmrir/Imagens e Matrizes da Tese de Thiago Alves Elias da Silva/Desenvolvimento da Metodologia/DOENTES/{}/Segmentadas".format(f)
    normal_dir_text = "../input/thermal-images-for-breast-cancer-diagnosis-dmrir/Imagens e Matrizes da Tese de Thiago Alves Elias da Silva/Desenvolvimento da Metodologia/DOENTES/{}/Matrizes".format(f)
    dir1 = os.path.join(normal_dir,"*.png")
    normal_files = glob.glob(dir1)
    dir2 = os.path.join(normal_dir_text,"*.txt")
    normal_files_text = glob.glob(dir2)

    try:
        for i in range(len(normal_files)):
            list1.append(normal_files[i]+"|"+normal_files_text[i])
    except:
        print(f)
            
    affected_files_text.extend(list1)
    
string2 = [1000,137,166,220,226,42,49,51,55,66,68,750,800,850,900]

healthy_files_text = []
for f in string2:
    list2 = []
    normal_dir = "../input/thermal-images-for-breast-cancer-diagnosis-dmrir/Imagens e Matrizes da Tese de Thiago Alves Elias da Silva/Desenvolvimento da Metodologia/SAUDA╠üVEIS/{}/Segmentadas".format(f)
    normal_dir_text = "../input/thermal-images-for-breast-cancer-diagnosis-dmrir/Imagens e Matrizes da Tese de Thiago Alves Elias da Silva/Desenvolvimento da Metodologia/SAUDA╠üVEIS/{}/Matrizes".format(f)
    dir1 = os.path.join(normal_dir,"*.png")
    normal_files = glob.glob(dir1)
    dir2 = os.path.join(normal_dir_text,"*.txt")
    normal_files_text = glob.glob(dir2)
    #print(len(normal_files),f)
    try:
        for i in range(len(normal_files)):
            list2.append(normal_files[i]+"|"+normal_files_text[i])
    except:
        print(f)
            
    healthy_files_text.extend(list2)
    

In [ ]:
string = [650,700]
affected_files_text2 = []
for f in string:
    list1 = []
    normal_dir = "../input/thermal-images-for-breast-cancer-diagnosis-dmrir/Imagens e Matrizes da Tese de Thiago Alves Elias da Silva/Desenvolvimento da Metodologia/DOENTES/{}/Segmentadas".format(f)
    normal_dir_text = "../input/thermal-images-for-breast-cancer-diagnosis-dmrir/Imagens e Matrizes da Tese de Thiago Alves Elias da Silva/Desenvolvimento da Metodologia/DOENTES/{}/Matrizes".format(f)
    dir1 = os.path.join(normal_dir,"*.png")
    normal_files = glob.glob(dir1)
    dir2 = os.path.join(normal_dir_text,"*.txt")
    normal_files_text = glob.glob(dir2)

    try:
        for i in range(len(normal_files)):
            list1.append(normal_files[i]+"|"+normal_files_text[i])
    except:
        print(f)
            
    affected_files_text2.extend(list1)
    
string2 = [950]

healthy_files_text2 = []
for f in string2:
    list2 = []
    normal_dir = "../input/thermal-images-for-breast-cancer-diagnosis-dmrir/Imagens e Matrizes da Tese de Thiago Alves Elias da Silva/Desenvolvimento da Metodologia/SAUDA╠üVEIS/{}/Segmentadas".format(f)
    normal_dir_text = "../input/thermal-images-for-breast-cancer-diagnosis-dmrir/Imagens e Matrizes da Tese de Thiago Alves Elias da Silva/Desenvolvimento da Metodologia/SAUDA╠üVEIS/{}/Matrizes".format(f)
    dir1 = os.path.join(normal_dir,"*.png")
    normal_files = glob.glob(dir1)
    dir2 = os.path.join(normal_dir_text,"*.txt")
    normal_files_text = glob.glob(dir2)
    #print(len(normal_files),f)
    try:
        for i in range(len(normal_files)):
            list2.append(normal_files[i]+"|"+normal_files_text[i])
    except:
        print(f)
            
    healthy_files_text2.extend(list2)
    

In [ ]:
affected_files_test.extend(affected_files_text2)
healthy_files_test.extend(healthy_files_text2)

In [ ]:
print(len(healthy_files_text))
print(len(affected_files_text))

In [ ]:
print(len(healthy_files_test))
print(len(affected_files_test))

In [ ]:
train_dic = {}
for f in affected_files_text:
  train_dic[f] = [1,0]
for f in healthy_files_text:
  train_dic[f] = [0,1]


test_dic = {}
for f in affected_files_test:
  test_dic[f] = [1,0]
for f in healthy_files_test:
  test_dic[f] = [0,1]

In [ ]:
import random
l_train = list(train_dic.items())
random.shuffle(l_train)


import random
l_test = list(test_dic.items())
random.shuffle(l_test)


# #ROBERTS AND SOBEL

In [ ]:
data = []
normal_data = []
labels = []
for i in range(len(l_train)):
    file_name,label = l_train[i]
    a,b = file_name.split("|")
    img = cv2.imread(a)
    arr_1 = np.empty((480,640))

    try:
        with open(b,"r") as f:
            arr = f.readlines()
            for i in range(len(arr)):
                arr[i] = arr[i].replace(",", ".")
                arr_2 = []
                for j in arr[i].split():
                    arr_2.append(float(j))
                arr_1[i] = np.asarray(arr_2) 
        img = np.asarray(img)
        arr_1 = np.asarray(arr_1)
        edge_roberts = roberts(arr_1)
        edge_sobel = sobel(arr_1)
        img = cv2.resize(img,(224,224),interpolation = cv2.INTER_CUBIC)
        arr_img = cv2.resize(arr_1,(224,224),interpolation = cv2.INTER_CUBIC)
        r_img = cv2.resize(edge_roberts,(224,224),interpolation = cv2.INTER_CUBIC)
        s_img = cv2.resize(edge_sobel,(224,224),interpolation = cv2.INTER_CUBIC)
        arr_3 = np.expand_dims(arr_img,axis = -1)
        arr_2 = np.expand_dims(r_img,axis = -1)
        arr_1 = np.expand_dims(s_img,axis = -1)
        img2 = np.concatenate((arr_3,arr_2,arr_1),axis = -1)
        height, width = img.shape[:2]
        img2 = img2.astype('float32')/255.0
        img = img.astype('float32')/255.0
        data.append(img2)
        normal_data.append(img)
        labels.append(label)

    except:
        print(i,file_name)
        print("Not possible")  
train_data = np.array(data)
print(train_data.shape)
normal_data = np.array(normal_data)
print(normal_data.shape)

train_labels = np.array(labels)
print(train_labels.shape)    

print('^_^-training data finished-^_^')

In [ ]:
test_text = []
data = []
labels = []
for i in range(len(l_test)):
    file_name,label = l_test[i]
    a,b = file_name.split("|")
    img = cv2.imread(a)
    arr_1 = np.empty((480,640))
    try:
        with open(b,"r") as f:
            arr = f.readlines()
            for i in range(len(arr)):
                arr_2 = []
                for j in arr[i].split():
                    arr_2.append(float(j))
                arr_1[i] = np.asarray(arr_2) 
        img = np.asarray(img)
        arr_1 = np.asarray(arr_1) 
        edge_roberts = roberts(arr_1)
        edge_sobel = sobel(arr_1) 
        img = cv2.resize(img,(224,224),interpolation = cv2.INTER_AREA)
        arr_img = cv2.resize(arr_1,(224,224),interpolation = cv2.INTER_CUBIC)
        s_img = cv2.resize(edge_roberts,(224,224),interpolation = cv2.INTER_CUBIC)
        r_img = cv2.resize(edge_sobel,(224,224),interpolation = cv2.INTER_CUBIC)
        arr_3 = np.expand_dims(arr_img,axis = -1)
        arr_2 = np.expand_dims(r_img,axis = -1)
        arr_1 = np.expand_dims(s_img,axis = -1)
        img2 = np.concatenate((arr_3,arr_2,arr_1),axis = -1)
        height, width = img.shape[:2]
        img2 = img2.astype('float32')/255.0
        img = img.astype('float32')/255.0
        test_text.append(img2)
        data.append(img)
        labels.append(label)

    except:
        print(i,file_name)
        print("Not possible")  
  
       
test_data = np.array(data)
print(test_data.shape)
test_text = np.array(test_text)
print(test_text.shape)
test_labels = np.array(labels)
print(test_labels.shape)    

print('^_^-testing data finished-^_^')

# **#CANNY AND roberts**

In [ ]:
from skimage.transform import rescale, resize, downscale_local_mean
data = []
normal_data = []
labels = []
for i in range(len(l_train)):
    file_name,label = l_train[i]
    a,b = file_name.split("|")
    img = cv2.imread(a)
    arr_1 = np.empty((480,640))

    try:
        with open(b,"r") as f:
            arr = f.readlines()
            for i in range(len(arr)):
                arr[i] = arr[i].replace(",", ".")
                arr_2 = []
                for j in arr[i].split():
                    arr_2.append(float(j))
                arr_1[i] = np.asarray(arr_2) 
        img = np.asarray(img)
        arr_1 = np.asarray(arr_1)
        edge_roberts = roberts(arr_1)
        edge_canny = feature.canny(arr_1, sigma=1)
        edge_canny = np.asarray(edge_canny)
        
        img = cv2.resize(img,(224,224),interpolation = cv2.INTER_CUBIC)
        arr_img = cv2.resize(arr_1,(224,224),interpolation = cv2.INTER_CUBIC)
        r_img = cv2.resize(edge_roberts,(224,224),interpolation = cv2.INTER_CUBIC)
        s_img = resize(edge_canny, (224,224),anti_aliasing=True)
        #s_img = cv2.resize(edge_canny,(224,224),interpolation = cv2.INTER_CUBIC)
        
        arr_3 = np.expand_dims(arr_img,axis = -1)
        arr_2 = np.expand_dims(r_img,axis = -1)
        arr_1 = np.expand_dims(s_img,axis = -1)
        img2 = np.concatenate((arr_3,arr_2,arr_1),axis = -1)
        height, width = img.shape[:2]
        img2 = img2.astype('float32')/255.0
        img = img.astype('float32')/255.0
        data.append(img2)
        normal_data.append(img)
        labels.append(label)

    except:
        print(i,file_name)
        print("Not possible")  
train_data = np.array(data)
print(train_data.shape)
normal_data = np.array(normal_data)
print(normal_data.shape)

train_labels = np.array(labels)
print(train_labels.shape)    

print('^_^-training data finished-^_^')

In [ ]:
test_text = []
data = []
labels = []
for i in range(len(l_test)):
    file_name,label = l_test[i]
    a,b = file_name.split("|")
    img = cv2.imread(a)
    arr_1 = np.empty((480,640))
    try:
        with open(b,"r") as f:
            arr = f.readlines()
            for i in range(len(arr)):
                arr_2 = []
                for j in arr[i].split():
                    arr_2.append(float(j))
                arr_1[i] = np.asarray(arr_2) 
        img = np.asarray(img)
        arr_1 = np.asarray(arr_1) 
        edge_roberts = roberts(arr_1)
        edge_canny = feature.canny(arr_1, sigma=1)
        edge_canny = np.asarray(edge_canny)
        img = cv2.resize(img,(224,224),interpolation = cv2.INTER_AREA)
        arr_img = cv2.resize(arr_1,(224,224),interpolation = cv2.INTER_CUBIC)
        s_img = cv2.resize(edge_roberts,(224,224),interpolation = cv2.INTER_CUBIC)
        r_img = resize(edge_canny, (224,224),anti_aliasing=True)
        #r_img = cv2.resize(edge_canny,(224,224),interpolation = cv2.INTER_CUBIC)
        arr_3 = np.expand_dims(arr_img,axis = -1)
        arr_2 = np.expand_dims(r_img,axis = -1)
        arr_1 = np.expand_dims(s_img,axis = -1)
        img2 = np.concatenate((arr_3,arr_2,arr_1),axis = -1)
        height, width = img.shape[:2]
        img2 = img2.astype('float32')/255.0
        img = img.astype('float32')/255.0
        test_text.append(img2)
        data.append(img)
        labels.append(label)

    except:
        print(i,file_name)
        print("Not possible")  
  
       
test_data = np.array(data)
print(test_data.shape)
test_text = np.array(test_text)
print(test_text.shape)
test_labels = np.array(labels)
print(test_labels.shape)    

print('^_^-testing data finished-^_^')

# **#CANNY AND prewitt**

In [ ]:
data = []
normal_data = []
labels = []
for i in range(len(l_train)):
    file_name,label = l_train[i]
    a,b = file_name.split("|")
    img = cv2.imread(a)
    arr_1 = np.empty((480,640))

    try:
        with open(b,"r") as f:
            arr = f.readlines()
            for i in range(len(arr)):
                arr[i] = arr[i].replace(",", ".")
                arr_2 = []
                for j in arr[i].split():
                    arr_2.append(float(j))
                arr_1[i] = np.asarray(arr_2) 
        img = np.asarray(img)
        arr_1 = np.asarray(arr_1)
        edge_prewitt = prewitt(arr_1)
        edge_canny = feature.canny(arr_1, sigma=1)
        edge_canny = np.asarray(edge_canny)
        img = cv2.resize(img,(224,224),interpolation = cv2.INTER_CUBIC)
        arr_img = cv2.resize(arr_1,(224,224),interpolation = cv2.INTER_CUBIC)
        r_img = cv2.resize(edge_prewitt,(224,224),interpolation = cv2.INTER_CUBIC)
        #s_img = cv2.resize(edge_canny,(224,224),interpolation = cv2.INTER_CUBIC)
        s_img = resize(edge_canny, (224,224),anti_aliasing=True)
        arr_3 = np.expand_dims(arr_img,axis = -1)
        arr_2 = np.expand_dims(r_img,axis = -1)
        arr_1 = np.expand_dims(s_img,axis = -1)
        img2 = np.concatenate((arr_3,arr_2,arr_1),axis = -1)
        height, width = img.shape[:2]
        img2 = img2.astype('float32')/255.0
        img = img.astype('float32')/255.0
        data.append(img2)
        normal_data.append(img)
        labels.append(label)

    except:
        print(i,file_name)
        print("Not possible")  
train_data = np.array(data)
print(train_data.shape)
normal_data = np.array(normal_data)
print(normal_data.shape)

train_labels = np.array(labels)
print(train_labels.shape)    

print('^_^-training data finished-^_^')

In [ ]:
test_text = []
data = []
labels = []
for i in range(len(l_test)):
    file_name,label = l_test[i]
    a,b = file_name.split("|")
    img = cv2.imread(a)
    arr_1 = np.empty((480,640))
    try:
        with open(b,"r") as f:
            arr = f.readlines()
            for i in range(len(arr)):
                arr_2 = []
                for j in arr[i].split():
                    arr_2.append(float(j))
                arr_1[i] = np.asarray(arr_2) 
        img = np.asarray(img)
        arr_1 = np.asarray(arr_1) 
        edge_prewitt = prewitt(arr_1)
        edge_canny = feature.canny(arr_1, sigma=1)
        edge_canny = np.asarray(edge_canny)
        img = cv2.resize(img,(224,224),interpolation = cv2.INTER_AREA)
        arr_img = cv2.resize(arr_1,(224,224),interpolation = cv2.INTER_CUBIC)
        s_img = cv2.resize(edge_prewitt,(224,224),interpolation = cv2.INTER_CUBIC)
        #r_img = cv2.resize(edge_canny,(224,224),interpolation = cv2.INTER_CUBIC)
        r_img = resize(edge_canny, (224,224),anti_aliasing=True)
        arr_3 = np.expand_dims(arr_img,axis = -1)
        arr_2 = np.expand_dims(r_img,axis = -1)
        arr_1 = np.expand_dims(s_img,axis = -1)
        img2 = np.concatenate((arr_3,arr_2,arr_1),axis = -1)
        height, width = img.shape[:2]
        img2 = img2.astype('float32')/255.0
        img = img.astype('float32')/255.0
        test_text.append(img2)
        data.append(img)
        labels.append(label)

    except:
        print(i,file_name)
        print("Not possible")  
  
       
test_data = np.array(data)
print(test_data.shape)
test_text = np.array(test_text)
print(test_text.shape)
test_labels = np.array(labels)
print(test_labels.shape)    

print('^_^-testing data finished-^_^')

# **Training Model**

In [ ]:
in_model = tf.keras.applications.DenseNet121(input_shape=(224,224,3),
                                               include_top=False,
                                               weights='imagenet',classes = 2)

inputs = tf.keras.Input(shape=(224,224,3))


x = in_model(inputs)
flat = Flatten()(x)

dense_1 = Dense(4096,activation = 'relu')(flat)

dense_2 = Dense(4096,activation = 'relu')(dense_1)

prediction = Dense(2,activation = 'softmax')(dense_2)

in_pred = Model(inputs = inputs,outputs = prediction)

29089792/29084464 [==============================] - 0s 0us/step


In [ ]:
in_pred.summary()
in_pred.compile(optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.0002), loss=tf.keras.losses.CategoricalCrossentropy(from_logits = False) , metrics=['accuracy'])
in_pred.fit(train_data,train_labels,batch_size = 32,epochs = 50)

In [ ]:
in_pred.evaluate(test_data,test_labels)

8/8 [==============================] - 0s 56ms/step - loss: 8.5441 - accuracy: 0.4647


[8.544113159179688, 0.46473029255867004]

In [ ]:
test_ = in_pred.predict(test_text)
Y_pred= np.argmax(test_labels, axis=1)
vgg19 = np.argmax(test_, axis=1)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
print(confusion_matrix(Y_pred,vgg19))
print(classification_report(Y_pred,vgg19))
print(accuracy_score(Y_pred,vgg19))